# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/home/travis/miniconda/envs/test/lib/python3.7/site-packages/IPython/core/display.py:701: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.20.0.98/12220/1 Dashboard: http://10.20.0.98:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,1062,Hannah,-0.4877428972510729,-0.25980060588909293
2000-01-01 00:00:01,1037,Quinn,0.6964831753475424,-0.33903039286667136
2000-01-01 00:00:02,972,Bob,-0.8168606366858395,0.41231546107682737
2000-01-01 00:00:03,1044,Kevin,-0.35514829258715586,0.5619118560789569
2000-01-01 00:00:04,1051,Bob,-0.41592271286018345,-0.8170343605736641
2000-01-01 00:00:05,1042,Ursula,-0.08979921713575711,-0.5913461783514455
2000-01-01 00:00:06,1003,Bob,-0.2683608675800122,-0.884634655882033
2000-01-01 00:00:07,964,Ray,-0.12518245915610704,-0.6034336301570047
2000-01-01 00:00:08,980,Yvonne,0.773738772041159,-0.14514144027244513


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1044,Michael,-0.2489052749598304,0.33543716463631834
2000-01-30 00:00:01,988,Alice,-0.06252437779291964,0.35713002967969376
2000-01-30 00:00:02,1019,Laura,-0.5328617864433491,0.4332252343810883
2000-01-30 00:00:03,1044,Kevin,0.10984499453282215,-0.008938485711312838
2000-01-30 00:00:04,980,Ursula,-0.3026346775491682,0.8303438148636295
2000-01-30 00:00:05,1032,Quinn,-0.21518655746426107,0.8582502167485659
2000-01-30 00:00:06,970,Yvonne,-0.024919144113811464,0.9492083010299317
2000-01-30 00:00:07,1028,Ray,0.35999336784666514,-0.9107599824091892
2000-01-30 00:00:08,965,Wendy,-0.6556311771404677,0.49524741200573663


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1062,Hannah,-0.487743,-0.259801
1,2000-01-01 00:00:01,1037,Quinn,0.696483,-0.339030
2,2000-01-01 00:00:02,972,Bob,-0.816861,0.412315
3,2000-01-01 00:00:03,1044,Kevin,-0.355148,0.561912
4,2000-01-01 00:00:04,1051,Bob,-0.415923,-0.817034


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,1062,Hannah,-0.487743,-0.259801
1,2000-01-01 00:00:01,1037,Quinn,0.696483,-0.339030
2,2000-01-01 00:00:02,972,Bob,-0.816861,0.412315
3,2000-01-01 00:00:03,1044,Kevin,-0.355148,0.561912
4,2000-01-01 00:00:04,1051,Bob,-0.415923,-0.817034


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.53 s, sys: 606 ms, total: 5.14 s
Wall time: 3.37 s


name
Alice       0.002871
Bob        -0.000681
Charlie     0.001027
Dan        -0.000781
Edith      -0.000985
Frank       0.000094
George      0.003767
Hannah     -0.000367
Ingrid     -0.000872
Jerry       0.000286
Kevin       0.000461
Laura       0.001181
Michael     0.000194
Norbert     0.001255
Oliver     -0.002983
Patricia    0.001322
Quinn      -0.000451
Ray        -0.000979
Sarah      -0.000536
Tim        -0.000040
Ursula      0.001285
Victor     -0.001847
Wendy       0.000551
Xavier     -0.000414
Yvonne      0.000084
Zelda      -0.002493
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.15.parquet  part.22.parquet  part.2.parquet
_metadata	  part.16.parquet  part.23.parquet  part.3.parquet
part.0.parquet	  part.17.parquet  part.24.parquet  part.4.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.5.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.6.parquet
part.12.parquet   part.1.parquet   part.27.parquet  part.7.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.8.parquet
part.14.parquet   part.21.parquet  part.29.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.42 s, sys: 165 ms, total: 1.58 s
Wall time: 1.07 s


name
Alice       0.002871
Bob        -0.000681
Charlie     0.001027
Dan        -0.000781
Edith      -0.000985
Frank       0.000094
George      0.003767
Hannah     -0.000367
Ingrid     -0.000872
Jerry       0.000286
Kevin       0.000461
Laura       0.001181
Michael     0.000194
Norbert     0.001255
Oliver     -0.002983
Patricia    0.001322
Quinn      -0.000451
Ray        -0.000979
Sarah      -0.000536
Tim        -0.000040
Ursula      0.001285
Victor     -0.001847
Wendy       0.000551
Xavier     -0.000414
Yvonne      0.000084
Zelda      -0.002493
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.27 s, sys: 84.1 ms, total: 1.35 s
Wall time: 946 ms


name
Alice       0.002871
Bob        -0.000681
Charlie     0.001027
Dan        -0.000781
Edith      -0.000985
Frank       0.000094
George      0.003767
Hannah     -0.000367
Ingrid     -0.000872
Jerry       0.000286
Kevin       0.000461
Laura       0.001181
Michael     0.000194
Norbert     0.001255
Oliver     -0.002983
Patricia    0.001322
Quinn      -0.000451
Ray        -0.000979
Sarah      -0.000536
Tim        -0.000040
Ursula      0.001285
Victor     -0.001847
Wendy       0.000551
Xavier     -0.000414
Yvonne      0.000084
Zelda      -0.002493
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html